**1) Proposition 1 -** Latest order date

**Functional Specification:**

<span style="color: var(--vscode-foreground);"><b>-</b>Use a subquery to find the MAX(OrderDate) from Sales.Orders.</span>

<span style="color: var(--vscode-foreground);">-Select all orders from Sales.Orders where OrderDate equals that maximum date.</span>

<span style="color: var(--vscode-foreground);">-Display OrderID, CustomerID, and OrderDate.</span>

<span style="color: var(--vscode-foreground);">-This identifies all orders made on the company’s latest active day.</span>

\*Quickly identifies the most recent day of business activity, which helps operations verify that orders are still being processed.

In [ ]:
SELECT 
    CustomerOrder.OrderID,
    CustomerOrder.OrderDate,
    CustomerOrder.CustomerID,
    CustomerOrder.SalespersonPersonID
FROM Sales.Orders AS CustomerOrder
WHERE CustomerOrder.OrderDate =
      (SELECT MAX(OrderDate)
       FROM Sales.Orders);

**2) Proposition 2 -** Identify the customer who placed the highest number of orders.

\-<span style="color: var(--vscode-foreground);">Use a subquery with GROUP BY and COUNT(*) to find which customer placed the highest number of orders</span><span style="color: var(--vscode-foreground);">.</span>

<span style="color: var(--vscode-foreground);">-</span><span style="color: var(--vscode-foreground);">Apply TOP (1) WITH TIES to include any customers tied for the top count.</span>

<span style="color: var(--vscode-foreground);">-Join the result back to Sales.Orders to display all order details for that customer.</span>

<span style="color: var(--vscode-foreground);">-Show CustomerID, OrderID, and OrderDate.</span>

<span style="color: var(--vscode-foreground);">*</span><span style="color: var(--vscode-foreground);">Lets marketing teams target high-frequency buyers for loyalty programs and promotions.</span>

In [ ]:
SELECT 
    CustomerOrder.CustomerID,
    CustomerOrder.OrderID,
    CustomerOrder.OrderDate
FROM Sales.Orders AS CustomerOrder
WHERE CustomerOrder.CustomerID IN (
    SELECT TOP (1) WITH TIES CustomerID
    FROM Sales.Orders
    GROUP BY CustomerID
    ORDER BY COUNT(*) DESC
);

**3) Proposition 3 -** Employees who did not place orders after may 2022

\-Select all employees from Application.People.

\-Use a NOT IN subquery on Sales.Orders to find employees whose IDs don’t appear in orders after '2022-05-01'.

\-Show each employee’s PersonID and FullName.

\-Optionally add their last order date using a subquery for reading

\*Helps managers track productivity and decide who might need additional support or reassignment.

In [ ]:
SELECT 
    Employee.PersonID,
    Employee.FullName
FROM Application.People AS Employee
WHERE Employee.PersonID NOT IN (
    SELECT CustomerOrder.SalespersonPersonID
    FROM Sales.Orders AS CustomerOrder
    WHERE CustomerOrder.OrderDate >= '2022-05-01'
);

**4) Proposition 4 -** List all orders placed on each customer’s last active day.

\-Use a correlated subquery to find the last order date for each customer from Sales.Orders.

\-Match the outer query’s CustomerID with the subquery’s CustomerID.

\-Return only the orders where the date equals the customer’s most recent order date.

\-Display CustomerID, OrderID, and OrderDate.

\*Reveals each customer’s last purchase date, helping retention teams identify when relationships may be going cold.

In [ ]:
SELECT
    CustomerOrder.CustomerID,
    CustomerOrder.OrderID,
    CustomerOrder.OrderDate
FROM Sales.Orders AS CustomerOrder
WHERE CustomerOrder.OrderDate = (
    SELECT MAX(PreviousOrder.OrderDate)
    FROM Sales.Orders AS PreviousOrder
    WHERE PreviousOrder.CustomerID = CustomerOrder.CustomerID
)
ORDER BY CustomerOrder.CustomerID;


**5) Proposition 5 -** <span style="color: var(--vscode-foreground);"> Stock items not ordered in 2015</span>  

\-Select StockItemID and StockItemName from Warehouse.StockItems.

\-Use a NOT EXISTS subquery to find items that were never ordered during 2015.

\-The subquery joins Sales.OrderLines and Sales.Orders to match OrderDate and StockItemID.

\-Filter OrderDate between '2015-01-01' and '2016-01-01'.

\*Identifies inventory that didn’t move during a given year so stock managers can plan markdowns or discontinue items.

In [ ]:
SELECT
    StockItem.StockItemID,
    StockItem.StockItemName
FROM Warehouse.StockItems AS StockItem
WHERE NOT EXISTS (
    SELECT 1
    FROM Sales.OrderLines AS OrderLine
    JOIN Sales.Orders AS CustomerOrder
      ON CustomerOrder.OrderID = OrderLine.OrderID
    WHERE OrderLine.StockItemID = StockItem.StockItemID
      AND CustomerOrder.OrderDate >= '2015-01-01'
      AND CustomerOrder.OrderDate <  '2016-01-01'
)
ORDER BY StockItem.StockItemName;

**6) Proposition 6 -** List orders that are not on the last day of the year

\-Create a CTE that adds a column for the end-of-year date using DATEFROMPARTS.

\-Compare each order’s OrderDate to the calculated year-end date.

\-Filter out any orders placed exactly on the last day of that year.

\-Display OrderID, OrderDate, and CustomerID.

\*Allows analysts to study sales trends while excluding year-end spikes that might distort averages.

In [ ]:
WITH YearOrders AS (
    SELECT 
        OrderID, 
        OrderDate,
        CustomerID,
        DATEFROMPARTS(YEAR(OrderDate), 12, 31) AS EndOfYear
    FROM Sales.Orders
)
SELECT OrderID, OrderDate, CustomerID
FROM YearOrders
WHERE OrderDate <> EndOfYear;

**7) Proposition 7 -** <span style="color: var(--vscode-foreground);"> Find the most recent order per employee.</span>  

<span style="color: var(--vscode-foreground);">-Join Sales.Orders to Application.People on SalespersonPersonID = PersonID.</span>

<span style="color: var(--vscode-foreground);">-Use a subquery that finds the MAX(OrderDate) for each SalespersonPersonID in Sales.Orders.</span>

\-Show SalespersonPersonID, FullName, OrderID, and OrderDate.

\*Shows which employees are actively selling, enabling managers to track recent performance.

In [ ]:
SELECT 
    o.SalespersonPersonID AS SalespersonPersonID,
    p.FullName,
    o.OrderID,
    o.OrderDate
FROM Sales.Orders AS o
JOIN Application.People AS p
  ON o.SalespersonPersonID = p.PersonID
WHERE o.OrderDate = (
    SELECT MAX(o2.OrderDate)
    FROM Sales.Orders AS o2
    WHERE o2.SalespersonPersonID = o.SalespersonPersonID
)
ORDER BY p.FullName;

**8) Proposition 8 -** Q2-2015 orders per salesperson (using a CTE)

\-Define a CTE named OrdersQ2\_2015 that filters Sales.Orders to include only orders placed between ‘2015-04-01’ and ‘2015-07-01’.

\-From the CTE, select SalespersonPersonID and OrderID to prepare data for aggregation.

\-Join the CTE to Application.People on SalespersonPersonID = PersonID to display each salesperson’s name.

\-Use COUNT to calculate how many orders each salesperson handled during Q2 of 2015.

\*Produces an easy quarterly performance view for leadership reviews and sales meetings.

In [ ]:
WITH OrdersQ2_2015 AS (
    SELECT
        SalespersonPersonID,
        OrderID
    FROM Sales.Orders
    WHERE OrderDate >= '2015-04-01'
      AND OrderDate <  '2015-07-01'
)
SELECT
    Salesperson.PersonID   AS SalespersonID,
    Salesperson.FullName   AS SalespersonName,
    COUNT(OrdersQ2_2015.OrderID) AS OrdersInQ2_2015
FROM OrdersQ2_2015
JOIN Application.People AS Salesperson
  ON Salesperson.PersonID = OrdersQ2_2015.SalespersonPersonID
GROUP BY Salesperson.PersonID, Salesperson.FullName
ORDER BY OrdersInQ2_2015 DESC;

**9) Proposition 9 -** Order totals via a derived table

\-Create a derived table (OrderTotals) that sums each order’s total by multiplying Quantity \* UnitPrice in Sales.OrderLines.

\-Group the derived table by OrderID to calculate one total per order.

\*Lets management track total revenue per order for profitability analysis.

In [ ]:
SELECT
    CustomerOrder.OrderID,
    CustomerOrder.OrderDate,
    OrderTotals.OrderTotal
FROM Sales.Orders AS CustomerOrder
JOIN (
    SELECT
        OrderID,
        SUM(Quantity * UnitPrice) AS OrderTotal
    FROM Sales.OrderLines
    GROUP BY OrderID
) AS OrderTotals
  ON OrderTotals.OrderID = CustomerOrder.OrderID
ORDER BY OrderTotals.OrderTotal DESC;

**10) Proposition 10 -** List orders and their total quantity using a derived table

\-Build a derived table (t) that calculates each order’s total quantity by summing the Quantity column from Sales.OrderLines.

\-Group the derived table by OrderID to get one total quantity per order.

\*Helps identify which orders move the highest volume of items, giving insight into large-scale customer demand.

In [15]:
SELECT 
    o.OrderID,
    o.OrderDate,
    t.TotalQuantity
FROM Sales.Orders AS o
JOIN (
    SELECT 
        OrderID,
        SUM(Quantity) AS TotalQuantity
    FROM Sales.OrderLines
    GROUP BY OrderID
) AS t
  ON o.OrderID = t.OrderID
ORDER BY t.TotalQuantity DESC;

(73595 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.299

OrderID,OrderDate,TotalQuantity
42479,2015-02-02,854
56910,2015-09-11,795
24890,2014-04-10,794
53643,2015-07-22,784
46297,2015-04-03,780
49643,2015-05-23,778
68751,2016-03-21,766
42025,2015-01-26,759
6849,2013-05-11,757
29868,2014-06-30,757
